## PART 1: Data Cleaning and Feature Engineering

In this notebook, I will walkthrough the process of inputting SET data file and generating a number of features, including the classification predictions.
The purpose is to get data ready for model building in the next part.

In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
import matplotlib.pyplot as plt

from datetime import datetime

In [42]:
# Read SET index price file & rename columns
# This file contains SET index from 2001 to the end of 2020 retrieved from Thomson Reuter Datastream
# Shape of this dataframe = (5220, 2)
# Properties: 1) Date (datetime64[ns]); 2) Close (float64)

SET_IDX_PRICE = pd.read_csv('data/SET/SET_Price.csv', parse_dates = True)
SET_IDX_PRICE = SET_IDX_PRICE.rename(columns = {'Name': 'Date', 'BNGKSET_P': 'Close'})
SET_IDX_PRICE['Date'] = pd.to_datetime(SET_IDX_PRICE['Date'])
SET_IDX_PRICE.head()

,Date,Close
0,2001-01-01,269.19
1,2001-01-02,269.19
2,2001-01-03,272.03
3,2001-01-04,278.75
4,2001-01-05,286.76


In [43]:
# Read SET index volume file & rename columns
# This file contains volume traded associated with SET index from 2001 to the end of 2020 retrieved from Thomson Reuter Datastream
# Shape of this dataframe = (5220, 2)
# Properties: 1) Date (datetime64[ns] ); 2) Volume (float64)

# Caveat: On some rows, the volume shows NaN because there is no trade occurred (for example, holidays)

SET_IDX_VOL = pd.read_csv('data/SET/SET_VO.csv', parse_dates = True)
SET_IDX_VOL = SET_IDX_VOL.rename(columns = {'Code': 'Date', 'BNGKSET(VO)': 'Volume'})
SET_IDX_VOL['Date'] = pd.to_datetime(SET_IDX_VOL['Date'])
SET_IDX_VOL.head()

,Date,Volume
0,2001-01-01,NaN
1,2001-01-02,NaN
2,2001-01-03,90861.75
3,2001-01-04,329329.70
4,2001-01-05,344688.40


In [47]:
# Time to merge the dataframe!
# Notice that the shape of df is now (5220, 3). That is great because we do not lose any observations from joining dfs
# In addition, I remove all the rows with missing volume (e.g. holidays)
# Finally, we get the SET_IDX of shape (4882, 3)

SET_IDX = pd.merge(SET_IDX_PRICE, SET_IDX_VOL, how='inner', on ='Date')
SET_IDX = SET_IDX.dropna()
SET_IDX = SET_IDX[SET_IDX['Volume'] > 0]
SET_IDX.head()

,Date,Close,Volume
2,2001-01-03,272.03,90861.75
3,2001-01-04,278.75,329329.70
4,2001-01-05,286.76,344688.40
5,2001-01-08,295.84,934483.00
6,2001-01-09,298.37,1361737.00


In [52]:
# Feature Generation 1: find 14-day EMA and EMA to extract the trend from the data
## Source: https://www.datacamp.com/community/tutorials/moving-averages-in-pandas
## Source: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html

SET_IDX['MA_14'] = SET_IDX.loc[:,'Close'].rolling(window=14).mean()
SET_IDX['EMA'] = SET_IDX.loc[:, 'Close'].ewm(span=19, adjust=False).mean()

SET_IDX.head()

,Date,Close,Volume,MA_14,EMA
2,2001-01-03,272.03,90861.75,NaN,272.030000
3,2001-01-04,278.75,329329.70,NaN,272.702000
4,2001-01-05,286.76,344688.40,NaN,274.107800
5,2001-01-08,295.84,934483.00,NaN,276.281020
6,2001-01-09,298.37,1361737.00,NaN,278.489918


In [75]:
SET_IDX['MA_5'] = SET_IDX.loc[:,'Close'].rolling(window=3).mean()
SET_IDX['MA_10'] = SET_IDX.loc[:,'Close'].rolling(window=3).mean()

In [53]:
# Feature Generation 2: Find MACD and RSI to account for the trend strength

EMA_12 = SET_IDX.loc[:, 'Close'].ewm(span=12, adjust=False).mean()
EMA_26 = SET_IDX.loc[:, 'Close'].ewm(span=26, adjust=False).mean()
SET_IDX['MACD'] = EMA_12 - EMA_26
SET_IDX.head()

,Date,Close,Volume,MA_14,EMA,MACD
2,2001-01-03,272.03,90861.75,NaN,272.030000,0.000000
3,2001-01-04,278.75,329329.70,NaN,272.702000,0.536068
4,2001-01-05,286.76,344688.40,NaN,274.107800,1.588930
5,2001-01-08,295.84,934483.00,NaN,276.281020,3.120045
6,2001-01-09,298.37,1361737.00,NaN,278.489918,4.485904


In [54]:
## Define a custom function for calculating RSI
## Source: https://stackoverflow.com/a/29400434

def RSI(close, window_length, SMA = False):
    ''' Arguments: close (Pandas series object, indicating the price inputted to calculate RSI)
                         window_length (the length needed to calculate simple moving average or input to the parameter of alpha in EMA)
                         SMA (if True, use SMA; otherwise, use EMA)
        Returns: Pandas series containing RSI (value between 0 to 100) except for the first entry outputting the string 'NaN'
                     because we lose one observation in calculating first-order difference in close
    '''
    
    delta = close.diff()
    # Take out the first row, be careful!
    delta = delta[1:]
    
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    
    
    if SMA == False:
        
        # Calculate Exponential Moving Average
        # Note that alpha = 2/(window_length + 1)
        roll_up1 = up.ewm(span=window_length).mean()
        roll_down1 = down.abs().ewm(span=window_length).mean()
        
        # Calculate the RSI based on EWMA
        RS1 = roll_up1 / roll_down1
        RSI1 = 100.0 - (100.0 / (1.0 + RS1))
        return pd.concat([pd.Series(['NaN']), RSI1])

    else:
        # Calculate the SMA
        roll_up2 = up.rolling(window_length).mean()
        roll_down2 = down.abs().rolling(window_length).mean()
        
        # Calculate the RSI based on SMA
        RS2 = roll_up2 / roll_down2
        RSI2 = 100.0 - (100.0 / (1.0 + RS2))
        return pd.concat([pd.Series(['NaN']), RSI2])

In [55]:
SET_IDX['RSI'] = RSI(SET_IDX.loc[:, 'Close'], 14, SMA = True)
SET_IDX.head()

,Date,Close,Volume,MA_14,EMA,MACD,RSI
2,2001-01-03,272.03,90861.75,NaN,272.030000,0.000000,NaN
3,2001-01-04,278.75,329329.70,NaN,272.702000,0.536068,NaN
4,2001-01-05,286.76,344688.40,NaN,274.107800,1.588930,NaN
5,2001-01-08,295.84,934483.00,NaN,276.281020,3.120045,NaN
6,2001-01-09,298.37,1361737.00,NaN,278.489918,4.485904,NaN


In [57]:
# Feature Generation 3: Momentum (use 1, 3, 5, 14 days)
SET_IDX['MOM1'] = SET_IDX.loc[:, 'Close'].pct_change(periods = 1)
SET_IDX['MOM3'] = SET_IDX.loc[:, 'Close'].pct_change(periods = 3)
SET_IDX['MOM5'] = SET_IDX.loc[:, 'Close'].pct_change(periods = 5)
SET_IDX['MOM14'] = SET_IDX.loc[:, 'Close'].pct_change(periods = 14)

SET_IDX.head()

,Date,Close,Volume,MA_14,EMA,MACD,RSI,MOM1,MOM3,MOM5,MOM14
2,2001-01-03,272.03,90861.75,NaN,272.030000,0.000000,NaN,NaN,NaN,NaN,NaN
3,2001-01-04,278.75,329329.70,NaN,272.702000,0.536068,NaN,0.024703,NaN,NaN,NaN
4,2001-01-05,286.76,344688.40,NaN,274.107800,1.588930,NaN,0.028735,NaN,NaN,NaN
5,2001-01-08,295.84,934483.00,NaN,276.281020,3.120045,NaN,0.031664,0.087527,NaN,NaN
6,2001-01-09,298.37,1361737.00,NaN,278.489918,4.485904,NaN,0.008552,0.070386,NaN,NaN


In [58]:
# Feature Generation 4: CCI
# Define custom function named CCI
def CCI(close, window_length, epsilon = 10**(-7)): 
    ''' This function only uses close price in calculating CCI, which could be an alternative version to (high + low + close)/3
        Arguments: close (Pandas series object, indicating the price inputted to calculate CCI)
                         window_length (the length needed to calculate simple moving average and standard deviation)
                         epsilon (prevent numerical problems where the denominator is very close to zero, default = 10^-7)
        Returns: Pandas series containing CCI
    '''
    CCI = pd.Series((close - close.rolling(window_length).mean()) / (epsilon + 0.015 * close.rolling(window_length).std())) 

    return CCI

In [62]:
SET_IDX['CCI_20'] = CCI(SET_IDX.loc[:, 'Close'], 20)
SET_IDX.head(5)

,Date,Close,Volume,MA_14,EMA,MACD,RSI,MOM1,MOM3,MOM5,MOM14,CCI_20
2,2001-01-03,272.03,90861.75,NaN,272.030000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
3,2001-01-04,278.75,329329.70,NaN,272.702000,0.536068,NaN,0.024703,NaN,NaN,NaN,NaN
4,2001-01-05,286.76,344688.40,NaN,274.107800,1.588930,NaN,0.028735,NaN,NaN,NaN,NaN
5,2001-01-08,295.84,934483.00,NaN,276.281020,3.120045,NaN,0.031664,0.087527,NaN,NaN,NaN
6,2001-01-09,298.37,1361737.00,NaN,278.489918,4.485904,NaN,0.008552,0.070386,NaN,NaN,NaN


In [67]:
SET_IDX.shift(1).head()

,Date,Close,Volume,MA_14,EMA,MACD,RSI,MOM1,MOM3,MOM5,MOM14,CCI_20
2,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2001-01-03,272.03,90861.75,NaN,272.03000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
4,2001-01-04,278.75,329329.70,NaN,272.70200,0.536068,NaN,0.024703,NaN,NaN,NaN,NaN
5,2001-01-05,286.76,344688.40,NaN,274.10780,1.588930,NaN,0.028735,NaN,NaN,NaN,NaN
6,2001-01-08,295.84,934483.00,NaN,276.28102,3.120045,NaN,0.031664,0.087527,NaN,NaN,NaN


In [68]:
# Feature Generation 5: On Balance Volume
# Define a custom function named OBV
## Source: https://stackoverflow.com/a/60093990
def OBV(df, close = 'close', volume = 'volume'):
    ''' Arguments: df (Dataframe that contains close column and volume column used to calculate OBV)
                         close (string indicating the column containing close price)
                         volume (string indicating the column containing the volume)
        Returns: Pandas series containing OBV
    '''
    series = np.where(df[close] > df[close].shift(1), df[volume], 
                                    np.where(df[close] < df[close].shift(1), -df[volume], 0)).cumsum()
    return series

In [69]:
SET_IDX['OBV'] = OBV(SET_IDX, close = 'Close', volume = 'Volume')
SET_IDX.head()

,Date,Close,Volume,MA_14,EMA,MACD,RSI,MOM1,MOM3,MOM5,MOM14,CCI_20,OBV
2,2001-01-03,272.03,90861.75,NaN,272.030000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,2001-01-04,278.75,329329.70,NaN,272.702000,0.536068,NaN,0.024703,NaN,NaN,NaN,NaN,329329.7
4,2001-01-05,286.76,344688.40,NaN,274.107800,1.588930,NaN,0.028735,NaN,NaN,NaN,NaN,674018.1
5,2001-01-08,295.84,934483.00,NaN,276.281020,3.120045,NaN,0.031664,0.087527,NaN,NaN,NaN,1608501.1
6,2001-01-09,298.37,1361737.00,NaN,278.489918,4.485904,NaN,0.008552,0.070386,NaN,NaN,NaN,2970238.1


In [89]:
%%HTML
<img src = "./Prediction_scheme.jpeg", width = 400 />

In [71]:
## Time to generate y: the prediction is separated into three classes:
## Up (1): if the price in prediction period (e.g. 1, 3, 5, 14 days) increases by more than 0.5% of today index
## Down (-1): if the price in prediction period (e.g. 1, 3, 5, 14 days) decreases by more than 0.5% of today index
## Sideway (0): if the price in prediction period (e.g. 1, 3, 5, 14 days) is between -0.5% to 0.5% of today index

# The number 0.5% comes from the paper "Performance of technical trading rules: evidence from Southeast Asian stock markets"
# written by Piyapas Tharavanij*, Vasan Siraprapasiri and Kittichai Rajchamaha

def generatePrediction(close, prediction_interval):
    ''' Arguments: close (Pandas series containing the close price)
        Returns: Pandas series containing true Y
    '''
    series = np.where(close.shift(-prediction_interval) > 1.005*close, 1, 
                                    np.where(close.shift(-prediction_interval) < 0.995*close, -1, 0))
    return series

In [76]:
SET_IDX['Y_1'] = generatePrediction(SET_IDX.loc[:, 'Close'], 1)
SET_IDX['Y_3'] = generatePrediction(SET_IDX.loc[:, 'Close'], 3)
SET_IDX['Y_5'] = generatePrediction(SET_IDX.loc[:, 'Close'], 5)
SET_IDX['Y_14'] = generatePrediction(SET_IDX.loc[:, 'Close'], 14)
SET_IDX.head(10)

,Date,Close,Volume,MA_14,EMA,MACD,RSI,MOM1,MOM3,MOM5,MOM14,CCI_20,OBV,Y_1,Y_3,Y_5,Y_14,MA_5,MA_10
2,2001-01-03,272.03,90861.75,NaN,272.030000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1,1,1,1,NaN,NaN
3,2001-01-04,278.75,329329.70,NaN,272.702000,0.536068,NaN,0.024703,NaN,NaN,NaN,NaN,329329.7,1,1,1,1,NaN,NaN
4,2001-01-05,286.76,344688.40,NaN,274.107800,1.588930,NaN,0.028735,NaN,NaN,NaN,NaN,674018.1,1,1,1,1,279.180000,279.180000
5,2001-01-08,295.84,934483.00,NaN,276.281020,3.120045,NaN,0.031664,0.087527,NaN,NaN,NaN,1608501.1,1,0,1,1,287.116667,287.116667
6,2001-01-09,298.37,1361737.00,NaN,278.489918,4.485904,NaN,0.008552,0.070386,NaN,NaN,NaN,2970238.1,-1,1,1,1,293.656667,293.656667
7,2001-01-10,293.65,684156.20,NaN,280.005926,5.128375,NaN,-0.015819,0.024027,0.079477,NaN,NaN,2286081.9,0,1,1,1,295.953333,295.953333
8,2001-01-11,295.06,734011.40,NaN,281.511334,5.685772,NaN,0.004802,-0.002637,0.058511,NaN,NaN,3020093.3,1,1,1,1,295.693333,295.693333
9,2001-01-12,311.25,1536920.00,NaN,284.485200,7.349193,NaN,0.054870,0.043168,0.085402,NaN,NaN,4557013.3,1,1,1,1,299.986667,299.986667
10,2001-01-15,317.51,1541838.00,NaN,287.787680,9.068066,NaN,0.020112,0.081253,0.073249,NaN,NaN,6098851.3,-1,-1,0,1,307.940000,307.940000
11,2001-01-16,308.42,800220.10,NaN,289.850912,9.586292,NaN,-0.028629,0.045279,0.033683,NaN,NaN,5298631.2,1,1,1,1,312.393333,312.393333


In [81]:
# Separate data files into Training, Validating, Test1, Test2
# Training covers the SET_IDX from 2008-2017, and we use data from 2009-2017 for training the model
# Validating covers the SET_IDX during 2018, and we use data to adjust hyperparameters
# Testing1 covers SET_IDX during 2019, and we use this to report our test result
# Testing2 covers SET_IDX during 2020, and we use this to do something more interesting such as track before and after Covid19

# DataShapes for Training, Validating, Testing_a and Testing_b are 2441 rows, 245 rows, 243 rows and 243 rows respectively.
# Caveat: We may not use all data from testing_b because at the end of the year, it uses future data not seen in dataset
Training_df = SET_IDX[(SET_IDX['Date'] > '2008-01-01') & (SET_IDX['Date'] < '2018-01-01')]
Validating_df = SET_IDX[(SET_IDX['Date'] > '2018-01-01') & (SET_IDX['Date'] < '2019-01-01')]
Testing_a_df = SET_IDX[(SET_IDX['Date'] > '2019-01-01') & (SET_IDX['Date'] < '2020-01-01')]
Testing_b_df = SET_IDX[(SET_IDX['Date'] > '2020-01-01') & (SET_IDX['Date'] < '2021-01-01')]

In [90]:
Training_df.to_csv('SET_Train.csv', index = False)
Validating_df.to_csv('SET_Validate.csv', index = False)
Testing_a_df.to_csv('SET_Test_a.csv', index = False)
Testing_b_df.to_csv('SET_Test_b.csv', index = False)

### That is the end of part 1. What we have initially is the raw SET index and SET volume traded data files retrieved from the datastream. Now, we just get the training, validating, and testing data ready to be used in model building